<a href="https://colab.research.google.com/github/leonardo201800478/StrifeFilterROM/blob/main/Filtragem_de_roms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import xml.etree.ElementTree as ET
import re
from collections import defaultdict
from google.colab import files

# Definição de prioridades de região
region_priority = {
    "BRA": 1,  # Brasil (Prioridade máxima)
    "PT": 2,   # Português
    "USA": 3,  # Estados Unidos
    "EUR": 4,  # Europa
    "JPN": 5   # Japão
}

# Expressão regular para detectar revisões (ex: "(Rev 1)", "(Rev 2)")
rev_pattern = re.compile(r"\(Rev (\d+)\)")

# Lista de palavras-chave que indicam versões digitais ou de teste
digital_variants = ["Sega Channel", "Sample", "Demo", "Beta", "Proto", "Prototype", "Digital", "Alpha", "Pirate", "[BIOS]", "[b]"]

# Lista de jogos que devem manter todas as revisões
specific_games = {
    "Revenge of Shinobi, The",
    "Castlevania III - Dracula's Curse",
    "Contra",
    "Teenage Mutant Ninja Turtles - Turtles in Time",
    "Mega Man",
    "Final Fight",
    "Street Fighter II",
    "Sonic the Hedgehog",
    "Super Mario Bros.",
    "Zelda II - The Adventure of Link"
}

# Função para carregar o arquivo .dat
def load_dat():
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    return filename

# Função para extrair o nome base do jogo (sem informações entre parênteses)
def get_base_name(name):
    return re.sub(r"\([^)]*\)", "", name).strip()

# Função para processar o .dat e aplicar os filtros
def process_dat(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    games_by_base = defaultdict(list)
    originals = {}

    # Itera sobre os jogos do .dat
    for game in root.findall("game"):
        name = game.get("name")
        clone_of = game.get("cloneof")
        description = game.find("description").text if game.find("description") else name

        # Verifica se a ROM é uma versão digital ou de teste
        is_digital = any(kw in name for kw in digital_variants)

        # Se for uma versão digital e tiver um cloneof, ignora
        if is_digital and clone_of:
            continue

        # Identifica a região do jogo
        releases = game.findall("release")
        regions = [r.get("region") for r in releases if r.get("region") in region_priority]
        primary_region = min(regions, key=lambda r: region_priority[r], default="OTH") if regions else "OTH"

        # Identifica a revisão
        revision_match = rev_pattern.search(name)
        revision = int(revision_match.group(1)) if revision_match else 0

        # Nome base do jogo (sem informações entre parênteses)
        base_name = get_base_name(name)

        # Adiciona ao dicionário agrupado pelo nome base
        games_by_base[base_name].append((game, primary_region, revision, clone_of))

        # Guarda as ROMs originais (sem cloneof)
        if not clone_of:
            originals[name] = game

    filtered_games = []

    for base_name, versions in games_by_base.items():
        # Ordena por prioridade de região e revisão (Prioridade > Última revisão)
        sorted_versions = sorted(versions, key=lambda x: (region_priority.get(x[1], float("inf")), -x[2]))

        # Mantém todas as revisões apenas para jogos da lista `specific_games`
        if base_name in specific_games:
            filtered_games.extend([v[0] for v in sorted_versions])
        else:
            # Mantém apenas a ROM de maior prioridade de região e revisão
            best_rom = sorted_versions[0][0]
            filtered_games.append(best_rom)

    # **Trata clones que devem ser mantidos (caso tenham uma região prioritária sobre o original)**
    for base_name, versions in games_by_base.items():
        for game, region, _, clone_of in versions:
            if clone_of and clone_of in originals:
                original_region = min(
                    [r.get("region") for r in originals[clone_of].findall("release") if r.get("region") in region_priority],
                    key=lambda r: region_priority[r],
                    default="OTH"
                )
                if region_priority.get(region, float("inf")) < region_priority.get(original_region, float("inf")):
                    filtered_games.append(game)

    return filtered_games

# Função para salvar e baixar o novo .dat filtrado
def save_filtered_dat(filtered_games, output_filename="Filtered_Sega_MegaDrive.dat"):
    new_root = ET.Element("datafile")
    for game in filtered_games:
        new_root.append(game)
    tree = ET.ElementTree(new_root)
    tree.write(output_filename, encoding="utf-8", xml_declaration=True)
    files.download(output_filename)

# Executar o processo
filename = load_dat()
filtered_games = process_dat(filename)
save_filtered_dat(filtered_games)

Saving Sega - Mega Drive - Genesis (Parent-Clone) (20250209-025231).dat to Sega - Mega Drive - Genesis (Parent-Clone) (20250209-025231) (17).dat


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>